# Python, Data, 2025

---

* [Agregace](),
* [Pivot tabulky](),
* [Časové řady](#Časové-řady),
    - [úvodní motivace](#Úvod),
    - [základní objekty](#Základní-objekty),
    - [indexování časovými údaji](#Indexování-s-časem),
    - [funkce to_datetime](#Funkce-TO_DATETIME),
    - [funkce date_range](#Funkce-DATE_RANGE),
    - [resampling](#Resampling),
    - [zpřesňování](#Zpřesňování)

---

## Agregace

---


<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.HtBtNx6dodlNptXV1QGHWQHaG0%26pid%3DApi&f=1&ipt=3a4d1dfa14a495127b4ece5cf689f00ed0c9e0836853358412e18161ae7c505f&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>


*Seskupování* neboli **agregace** jsou procesy, které patří k základní efektivní analýze dat.

### Jednoduchá agregace

---

Přesto, že veškeré základní **statistické údaje** prakticky nabízí metoda `describe()`, můžeš ocenit, když stejnou statistiku můžeš aplikovat **na tebou vybrané objekty**.

Mezi nejjednodušší postupy, jak data analyzovat patří metody jako:
* `sum()`,
* `mean()`,
* `median()`,
* `min()`,
* `max()`.

Všechny tyto metody umožní získat jedno samotné číslo, které ti umožní prohlédnout podstatu zadaného datasetu.

In [ ]:
from pandas import read_csv, DataFrame

In [ ]:
nemovitosti_df = read_csv("../onsite/housing.csv")

In [ ]:
nemovitosti_df.head()

In [ ]:
nemovitosti_df.info()

In [ ]:
nemovitosti_df.describe()

<br>

Prvně vyberu sloupec, se kterým chci pracovat:

In [ ]:
nemovitosti_df.loc[:, "price"]

In [ ]:
nemovitosti_df.loc[:, "price"].sum()

<br>

Často ale není dostačující, prozkoumat data pouze jednoduchých agregačních funkcí.

Další operace, které je potřeba pochopit jsou seskupování dat podle zadaných parametrů.

<br>

### Složitější agregace, seskupování GROUPBY

---

Metoda `groupby`, původně operace z SQL jazyka, je v rámci knihovny `pandas` všestraný pomocník pro seskupování dat na základě různých kritérií.

In [ ]:
from pandas import DataFrame

In [ ]:
pokus_s_cisly_df = DataFrame(
    {'klíč': ['A', 'B', 'C', 'A', 'B', 'C'],
     'číselná hodnota': range(6)},
)

In [ ]:
pokus_s_cisly_df

#### Klasické seskupení podle sloupečku

---

In [ ]:
# pokus_s_cisly_df.groupby?

In [ ]:
pokus_s_cisly_df.groupby("klíč")

Jde opět o tzv. *lazy evaluation* proces, samotný nic neprovede, pouze čeká na pokyn uživatele, který samotnou agregaci spustí.

In [ ]:
type(pokus_s_cisly_df.groupby("klíč"))

Metoda standardně vrací `DataFrameGroupBy`.

In [ ]:
pokus_s_cisly_df

In [ ]:
pokus_s_cisly_df.groupby("klíč").max()
# max_pokus_s_cisly
# count_pokus_s_cisly

Metoda `sum()` je pouze jednou z možností, se kterou můžeš pracovat.

<br>

#### Sloupečkové označování

---

Stejně jako `DataFrame` můžeš označovat také *GroupBy* objekty.

In [ ]:
pokus_s_cisly_df

In [ ]:
pokus_s_cisly_df.groupby("klíč")

In [ ]:
pokus_s_cisly_df.groupby("klíč")["číselná hodnota"]

In [ ]:
pokus_s_cisly_df.groupby("klíč")["číselná hodnota"].sum()

<br>

V uplynulé ukázce je zadaná seskupování podle sloupečku `klíč`.

Dále je vybraný pouze konkrétní sloupeček, na který chceš spustit metodu `sum`.

<br>

Pokud potřebuješ nad vybraným objektem provádět některé procesy ručně, můžeš přes *GroupBy* objekt **iterovat**:

In [ ]:
for (klic, hodnota) in pokus_s_cisly_df.groupby("klíč"):
    print(f"Klic: {klic}; Hodnota={hodnota}")

<br>

### Agregace s metodami

---

Kromě jednoduchých agregací, nabízí *GroupBy* řadu další funkcionality.

Jde o metody:
* `aggregate`,
* `filter`,
* `transform`,
* `apply`.

#### Metoda AGGREGATE

---

In [ ]:
import numpy

In [ ]:
rng = numpy.random.RandomState(0)

In [ ]:
from pandas import DataFrame

In [ ]:
pokus_s_cisly_df = DataFrame(
    {'klíč': ['A', 'B', 'C', 'A', 'B', 'C', 'C'],
     'data_1': range(7),
     'data_2': rng.randint(0, 10, 7)}
)

In [ ]:
pokus_s_cisly_df

Statistické metody určitě nabízí spousty pomůcek.

Objekt typu *GroupBy* umí pracovat také s funkcemi, stringy a celými listy.

In [ ]:
pokus_s_cisly_df.groupby("klíč").aggregate("sum")  # .sum()

In [ ]:
pokus_s_cisly_df.groupby("klíč").aggregate(numpy.median)

In [ ]:
pokus_s_cisly_df.groupby("klíč").aggregate(["min", numpy.median, max])

<br>

Metoda `aggregate`umožní vytvořit tzv. *podsloupce*.

Také umožní elegantně spojit různé matematické funkce (př. `min`, `"min"`, `numpy.min`).

<br>

Pokud neuložím nově seskupenou tabulku/sloupec, potom data neuchovám.

Metoda `groupby` **neupravuje zdrojová data**.

In [ ]:
pokus_s_cisly_df

Můžeš říct, že metoda `aggregate` se používá k aplikaci **jedné nebo více agregačních funkcí na seskupená data**.

| pobocka | ovoce | prodane_mnozstvi | cena |
| :- | :- | :- | :- |
| A | apple | 10 | 20 |
| A | banana | 15 | 12 |
| B | apple | 8 | 22 |
| B | banana | 30 | 10 |
| C | apple | 20 | 18 |
| C | banana | 25 | 15 |

In [ ]:
ovoce_data = {
    'pobocka': ['A', 'A', 'B', 'B', 'C', 'C'],
    'ovoce': ['jablko', 'banan', 'jablko', 'banan', 'jablko', 'banan'],
    'prodane_mnozstvi': [10, 15, 8, 30, 20, 25],
    'cena': [20, 12, 22, 10, 18, 15]
}

In [ ]:
ovoce_data.keys()

In [ ]:
ovoce_df = DataFrame(ovoce_data)

In [ ]:
ovoce_df.head(6)

In [ ]:
ovoce_df.groupby('pobocka').aggregate({'prodane_mnozstvi': ["min", numpy.median, max]})

Pro jednodušší analýzu stačí vybrat sloupec:

In [ ]:
ovoce_df.groupby('pobocka')['prodane_mnozstvi'].sum()

<br>

Je nutné vždy ověřit výstup po agregaci, jestli jsem správně vybral metody:

In [ ]:
ovoce_df.groupby('pobocka').sum()

In [ ]:
ovoce_df.groupby('pobocka').aggregate({'prodane_mnozstvi': sum, 'cena': 'mean'})

1. Nejprve jsou hodnoty seskupení podle sloupce `pobocka`,
2. poté specifikuješ pomocí `aggregate` funkce a sloupce,
3. .. tedy sumarizovat hodnoty v `prodane_mnozstvi` a získat průměr `cena` pro každou pobočku.

### Metoda FILTER

---

Filtrování ti umožní zahodit takové údaje, které nesplňují zadanou podmínku.

Metoda `filter` se používá k vybrání **seskupených dat** podle splnění **určité podmínky**.

Představme si, že máme následující dataset s informacemi o prodeji ovoce v různých obchodech:

```
pobocka,ovoce,prodane_mnozstvi,cena
A,apple,10,20
A,banana,15,12
B,apple,8,22
B,banana,30,10
C,apple,20,18
C,banana,25,15
```

In [ ]:
ovoce_data = {
    'pobocka': ['A', 'A', 'B', 'B', 'C', 'C'],
    'ovoce': ['jablko', 'banan', 'jablko', 'banan', 'jablko', 'banan'],
    'prodane_mnozstvi': [10, 15, 8, 30, 20, 25],
    'cena': [20, 12, 22, 10, 18, 15]
}

In [ ]:
ovoce_df = DataFrame(ovoce_data)

In [ ]:
ovoce_df

In [ ]:
    def vyber_s_prodejem_nad_limit(data, limit=30):
        return data['prodane_mnozstvi'].sum() > limit

<br>

Funkce `vyber_s_prodejem_nad_limit`, tedy **filtrovací funkce**, musí vracet **boolean** datový typ.

In [ ]:
vyfiltrovane_df = ovoce_df.groupby('pobocka').filter(vyber_s_prodejem_nad_limit)

In [ ]:
vyfiltrovane_df.groupby('pobocka')['prodane_mnozstvi'].sum()

1. Nejprve seskupíš data podle sloupce `pobocka` pomocí `groupby`,
2. dále použiješ metodu `filter`, která umožňuje použít uživatelem definovanou funkci (`vyber_pouze_pobocky_s_prodejem_nad_limit`),
3. definuješ funkci, která vybere pouze pobočky s větším prodejem než je parametr `limit`,
4. metoda `filter` pak vybere pouze ty řádky, které splňují tuto podmínku.

In [ ]:
ovoce_df[ovoce_df['prodane_mnozstvi'] > 19]  # Boolean porovnávání

### Metoda TRANSFORM

---

Zatímco předchozí výsledky *agregace* pomocí `groupby` vraceli redukované množství dat.

Transformace obvykle vrací data o stejném rozsahu jako vstupní data. Jenom upravená.

Metoda `transform` se používá k aplikaci určité **transformační funkce** na každý prvek seskupených dat.

In [ ]:
ovoce_df

In [ ]:
def vrat_procenta_z_celkoveho_prodeje(udaj):
    return round(udaj / udaj.sum() * 100, 1)

<br>

Vytvoření nového sloupečku `vrat_procenta_z_celkoveho_prodeje`:

In [ ]:
ovoce_df['Procento z celkoveho prodeje'] = \
    ovoce_df.groupby('pobocka')['prodane_mnozstvi'].transform(
        vrat_procenta_z_celkoveho_prodeje
    )

In [ ]:
# ovoce_df

1. Nejprve seskupíš data podle sloupce `pobocka` pomocí `groupby`,
2. dále použiješ metodu `transform`, která umožňuje použít uživatelem definovanou funkci,
3. definuješ funkci, která vybere vypočítá procentuální vyjádření prodaného zboží pro pobočku,
4. přidáš nový sloupeček `'Procento z celkoveho prodeje'`.

### Metoda `APPLY`

---

Metoda `apply` ti také dovolí, používat uživatelem definované funkce na seskupená data.

Následně vrací objekt knihovny `pandas` (buď `DataFrame`, nebo `Series`, a nebo skalární hodnota).

In [ ]:
ovoce_df

In [ ]:
def vypocitej_vydelek_za_artikl(seskupene):
    seskupene['vydelek_za_artikl'] = seskupene['prodane_mnozstvi'] * seskupene['cena']
    return seskupene

<br>

Metoda `apply` je **obecnější a flexibilnější** než `transform`.

`apply` umožňuje použít uživatelem definovanou funkci na každou skupinu po seskupení dat pomocí `groupby`.

Výsledek metody `apply` může mít jiný tvar než původní data.

In [ ]:
ovoce_df.groupby('pobocka').apply(vypocitej_vydelek_za_artikl)

In [ ]:
ovoce_df.groupby('pobocka', group_keys=False).apply(vypocitej_vydelek_za_artikl)

In [ ]:
# ovoce_df.groupby?

Od posledních verzí frameworku (`1.5.0` a vyšší) platí, že pokud bude výsledkem `DataFrame` nebo `Series` musíš uvést argument pro `group_keys=True`).

<br>

In [ ]:
vystup = ovoce_df.groupby('pobocka', group_keys=True).apply(vypocitej_vydelek_za_artikl)

In [ ]:
vystup

In [ ]:
ovoce_df

Na první pohled vypadají metody `apply` a `transform` docela podobně.

Metoda `transform` je trochu omezenější než `apply`.

Slouží k aplikaci uživatelem definované nebo vestavěné funkce **na každý prvek** skupiny po seskupení s `groupby`.

`transform` musí vracet hodnotu stejného tvaru jako vstupní data.

### Souhrn k metodám APPLY a TRANSFORM

---

Výsledek metody `transform` **má stejný tvar jako původní data**.

#### Apply
* potřebuješ výsledek, který **má jiný tvar než původní data**,
* umí zpracovat **více sloupečků současně**.

#### Transform
* potřebuješ výsledek, který **má stejný tvar jako původní data**,
* umí zpracovat **pouze jeden sloupeček**.

In [ ]:
df_rozdily = DataFrame({
    'KLIC': ['A','B','C'] * 3,
    'A': numpy.arange(9),
    'B': [1,2,3] * 3,
})

In [ ]:
df_rozdily

#### `transform` vrací výsledky ve stejném tvaru

---

In [ ]:
def vypocitej_sumu(data):
    return data.sum()

In [ ]:
seskup_df_rozdily_apply = df_rozdily.groupby('KLIC')['A'].apply(vypocitej_sumu)

In [ ]:
seskup_df_rozdily_apply

In [ ]:
seskup_df_rozdily_trans = df_rozdily.groupby('KLIC')['A'].transform(vypocitej_sumu)

In [ ]:
seskup_df_rozdily_trans

#### `apply` umí pracovat s více sloupečky, `transform` jen s jedním

---

In [ ]:
df_rozdily

In [ ]:
def vypocitej_rozdil(data):
    return data['B'] - data['A']

In [ ]:
df_rozdily.groupby('KLIC').apply(vypocitej_rozdil)

In [ ]:
# df_rozdily.groupby('KLIC').transform(vypocitej_rozdil)

---

<br>

**🧠 CVIČENÍ 🧠, procvič si funkcí GroupBy a agregační funkce**

Máš zadaný takový datový set.

Následně:
1. Pomocí metody `filter` vyber prodejny, které prodaly **alespoň 30 produktů**,
2. na filtrovaném datasetu použijte metodu `apply` pro výpočet **celkového příjmu z prodeje pro každý obchod**.

In [ ]:
from pandas import DataFrame

In [ ]:
df_prodej_hardware = DataFrame({
    'prodejna_id': (5, 4, 1, 5, 5, 1, 4, 2, 5, 1, 3, 1, 3, 4, 2, 1, 5, 4, 1, 5),
    'transakce_id': (1278, 1216, 1866, 1872, 1797, 1272, 1880, 1061, 1595, 1879, 1728,
       1341, 1396, 1698, 1018, 1176, 1611, 1395, 1444, 1232),
    'predmet_prodeje': ('grafická_karta', 'SSD', 'RAM', 'procesor', 'grafická_karta',
       'základní_deska', 'SSD', 'SSD', 'grafická_karta', 'RAM',
       'grafická_karta', 'procesor', 'grafická_karta', 'SSD',
       'grafická_karta', 'RAM', 'základní_deska', 'HDD', 'grafická_karta',
       'RAM'),
    'pocet_prodanych_ks': (1,  5,  6,  6,  3,  7,  9, 10,  8,  6,  8,  5,  8, 10,  4, 10,  8,
       10,  2,  5),
    'cena_predmetu': (19500.69874949, 19731.10951735, 14114.15342339, 10953.87914371,
        6535.78851758, 16369.00288429, 13852.2578648 ,  3671.03031723,
       18263.08009763, 16539.476237  , 19021.09830919, 14651.53041357,
       12461.59632075,  8655.73920767, 18688.2054254 , 17388.24584526,
        1381.76406707,  1014.1560027 ,  7841.03565412, 16305.78995025)
})

In [ ]:
df_prodej_hardware.head(7)

In [ ]:
df_prodej_hardware.groupby('prodejna_id')['pocet_prodanych_ks'].sum()

In [ ]:
# S filtrem:
def vyber_prodejny_s_prodejem_pres_tricet_ks(data_prodej,
                                             jmeno_sl='pocet_prodanych_ks',
                                             limit=30):
    if jmeno_sl not in data_prodej.columns:
        return None
    else:
        return data_prodej[jmeno_sl].sum() > limit

In [ ]:
vyfiltr_df = df_prodej_hardware.groupby('prodejna_id').filter(vyber_prodejny_s_prodejem_pres_tricet_ks)

In [ ]:
vyfiltr_df.head()

In [ ]:
vyfiltr_df['predmet_prodeje'].unique()

In [ ]:
vyfiltr_df.groupby('prodejna_id')

In [ ]:
df_prodej_hardware.head(2)

In [ ]:
# Bez filtru:

In [ ]:
seskupene_prodeje = df_prodej_hardware.groupby('prodejna_id')['pocet_prodanych_ks'].sum()

In [ ]:
type(seskupene_prodeje)

In [ ]:
seskupene_prodeje[seskupene_prodeje > 30]

In [ ]:
def vypocti_celkovy_prijem_obchodu(df_prodej_hardware):
    return round(df_prodej_hardware['pocet_prodanych_ks']
                 * df_prodej_hardware['cena_predmetu'], 2)

In [ ]:
vyfiltr_df['Celkové prodeje'] = \
    vyfiltr_df.groupby('prodejna_id', group_keys=False).apply(vypocti_celkovy_prijem_obchodu)

In [ ]:
type(vyfiltr_df.groupby('prodejna_id')['Celkové prodeje'].sum())

<br>

Pokud bude potřebné, seřadit sestupně podle celkového výdělku prodejny:

In [ ]:
vyfiltr_df \
    .groupby('prodejna_id')['Celkové prodeje'] \
    .sum() \
    .sort_values(ascending=False)

<details>
    <summary>▶️ Řešení</summary>
    
```python
def vyber_prodejny_s_vice_nez_triceti_produkty(
    data: DataFrame, limit: int = 30
):
    return data['pocet_prodanych_ks'].sum() >= limit

hledane_prodejny = df_prodej_hardware.groupby('prodejna_id').filter(
    vyber_prodejny_s_vice_nez_triceti_produkty
)

agregovano = df_prodej_hardware.groupby('prodejna_id')['pocet_prodanych_ks'].sum()
odfiltrovano = agregovano[agregovano > 30]

print(odfiltrovano)
```
</details>

---


<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse2.mm.bing.net%2Fth%3Fid%3DOIP.P16hnt14CM-mJ5eAKykxLAHaHa%26pid%3DApi&f=1&ipt=27e3f5b23fe44567ad0730192d5dcb13c7c23aa6e87579d16df05e27461dc12c&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

## Pivot tabulky

---


*Pivot tabulky* (*kontingenční tabulky*) jsou užitečné pro přehledné zobrazení a analýzu dat z tabulkových zdrojů.

Pomocí pivot tabulek můžeš **seskupit data podle určitých kategorií** a provést agregaci hodnot.

Nejprve si představ situaci bez pivot tabulek, pomocí ukázky níže:

In [ ]:
# !pip install seaborn

In [ ]:
import seaborn

<br>

V této ukazce použiješ vzorová data týkající se [nehody lodi Titanic](https://en.wikipedia.org/wiki/Sinking_of_the_Titanic):

In [ ]:
df_titanic = seaborn.load_dataset('titanic')

In [ ]:
df_titanic.info()

<br>

Pro jakoukoliv pokročilou analýzu dat, potřebuješ údaj seskupit.

Tak jak jsi do prováděl doposud, pomocí `groupby`:

In [ ]:
df_titanic.head()

<br>

Pokud potřebuješ identifikovat všechny unikátní hodnoty ve sloupci, vyber sloupec a použij metodu `unique()`:

In [ ]:
df_titanic["sex"].unique()

<br>

Pro následující rozbor seskupím data podle pohlaví:

In [ ]:
df_titanic.groupby('sex')

<br>

Potřebuješ průměrnou hodnotu, ať je na sloupečku `survived` vidět poměr přeživších:

In [ ]:
df_titanic.head(3)

In [ ]:
df_titanic.groupby('sex')['survived'].mean()

<br>

Pro lepší čitelnost **zaokrouhlím výsledek na dvě desetinné čísla**:

In [ ]:
# round(df_titanic.)

In [ ]:
df_titanic.groupby('sex')['survived'].mean().round(2)

Zaokrouhlit je možné nejenom pomocí Pythonu a funkce `round()` ale také metoda `round()`.

Takový průzkum z této studie ti dá jasný pohled na věc:
* 3 ze 4 žen přežily,
* 1 z 5 mužů přežil.

<br>

Pokud budeš potřebovat detailnější analýzy, budeš potřebovat více dat.

Třeba situaci, kde kromě pohlaví, bereš v potaz **třídu cestujících** `class`:

In [ ]:
df_titanic.head(2)

In [ ]:
df_titanic["class"].unique()

In [ ]:
type(df_titanic.groupby(['sex', 'class'])['survived'].mean())

In [ ]:
type(df_titanic.groupby(['sex', 'class'])[['survived']].mean())

In [ ]:
df_titanic.groupby(['sex', 'class'])[['survived']].mean()

In [ ]:
df_titanic.groupby(['sex', 'class'])[['survived']].mean().round(2)

<br>

Metodou `unstack` si můžeš vytvořit nové sloupečky, které jsou postavené na novém Indexu, nebo Indexech (*Multiindex*):

In [ ]:
df_titanic.groupby(['sex', 'class'])['survived'].mean().round(2).unstack()

Takový průzkum ti dá skutečně lepší pohled na věc.

Současně ale roste **náročnost ohlášení**.

Zápis "bobtná" a **stává se náročnějším na přečtení a pochopení**.

<br>

### Pivot tabulka

---

Podobné řešení ti nabízí funkce `pivot_table`:

In [ ]:
df_titanic.groupby(['sex', 'class'])['survived'].mean().round(2).unstack()

In [ ]:
df_titanic.pivot_table('survived', index='sex', columns='class').round(2)

In [ ]:
pivot_df = df_titanic.pivot_table('survived', index='sex', columns='class').round(2)

In [ ]:
df_titanic.index

In [ ]:
pivot_df.index

Zásádním rozdílem je ovšem **čitelnost**, kterou máš pro tuto variantu zápisu.

```
Clarity beats purity
```

Pomocí vhodných argumentů, můžeš doplnit vysvětlivky tam, kde funkce `groupby` nemohla.

Stejně platí, že pokud budeš potřebovat **další Index**, můžeš si pomoci funkcí `cut`:

In [ ]:
df_titanic.head(2)

In [ ]:
df_titanic['age'].describe()

In [ ]:
from pandas import cut

In [ ]:
age = cut(df_titanic['age'], [0, 40, 60, 70])

In [ ]:
age  # < xxx , xxx )

In [ ]:
vystup_vek_df = df_titanic.pivot_table('survived',
                                       index=['sex', age],
                                       columns='class').round(2)

In [ ]:
vystup_vek_df

In [ ]:
vystup_vek_df.index

Pomocí funkce `cut` doplníš tabulku na **MultiIndex**.

Ten nyní vytvoří i rozsah pro věkovou kategorii.

<br>

#### 🧠 CVIČENÍ 🧠, procvič si pivot tabulky

---

Ze zadaného datasetu vytvoř **pivot tabulku**, která zobrazí **počet prodaných kusů ovoce** pro **každý obchod** a **druh ovoce**.

In [ ]:
ovoce_data = {
    'pobocka': ['A', 'A', 'B', 'B', 'C', 'C'],
    'ovoce': ['jablko', 'banan', 'jablko', 'banan', 'jablko', 'banan'],
    'prodane_mnozstvi': [10, 15, 8, 30, 20, 25],
    'cena': [20, 12, 22, 10, 18, 15]
}

In [ ]:
# pivot_table()
#              prodane_mnozstvi      jablko banan
# Pobocka
#    A 
#    B
#    C 

In [ ]:
ovoce_data_df = DataFrame(ovoce_data)

In [ ]:
ovoce_data_df['ovoce'].unique()

In [ ]:
ovoce_data_df.head()

In [ ]:
ovoce_data_df.pivot_table('prodane_mnozstvi', index='pobocka', columns='ovoce')

In [ ]:
ovoce_data_df['cena'].min()

In [ ]:
cenovy_intervaly = cut(ovoce_data_df['cena'],
                       [ovoce_data_df['cena'].min(),
                        15, ovoce_data_df['cena'].max()], include_lowest=True)

In [ ]:
cenovy_intervaly

In [ ]:
ovoce_data_df.pivot_table('prodane_mnozstvi',
                          index=['pobocka', cenovy_intervaly], columns='ovoce')

In [ ]:
ovoce_data_df

<details>
    <summary>▶️ Řešení</summary>
    
```python
pivot_tabulka = df_ovoce.pivot_table(values="prodane_mnozstvi", index="pobocka", columns="ovoce")
```
</details>

In [ ]:
# !pip install openpyxl

In [ ]:
from pandas import read_excel

In [ ]:
data_df = read_excel('../onsite/RESULT_LIST_ANONYM_CAPACITY_MARKET_FCR_2025-05-20_2025-05-20.xlsx')

In [ ]:
data_df.info()

In [ ]:
data_df.head(3)

#### 1. Převedeme data ve sloupcích

---

In [ ]:
data_df.columns

In [ ]:
soucasne_poradi_sloupcu = tuple(data_df.columns)

In [ ]:
soucasne_poradi_sloupcu

In [ ]:
nove_poradi_sloupcu = (
     'DATE_FROM',
     'DATE_TO',
     'TYPE_OF_RESERVES',
     'PRODUCT',
     'OFFERED_CAPACITY_PRICE_[EUR/MW]',
     'OFFERED_CAPACITY_PRICE_NEW',
     'OFFERED_CAPACITY_[MW]',
     'ALLOCATED_CAPACITY_[MW]',
     'COUNTRY',
     'SETTLEMENTCAPACITY_PRICE_[EUR/MW]',
     'SETTLEMENTCAPACITY_PRICE_NEW',
     'NOTE'
)

In [ ]:
data_df['OFFERED_CAPACITY_PRICE_NEW'] = data_df['OFFERED_CAPACITY_PRICE_[EUR/MW]'] / 4

In [ ]:
data_df['SETTLEMENTCAPACITY_PRICE_NEW'] = data_df['SETTLEMENTCAPACITY_PRICE_[EUR/MW]'] / 4

In [ ]:
data_df.tail(2)

<br>

Pro změnu pořadí sloupců, nachystáme novou tabulku, s novým pořadím:

In [ ]:
from pandas import DataFrame

In [ ]:
novy_df = DataFrame(data_df, columns=nove_poradi_sloupcu)

In [ ]:
novy_df.tail(2)

#### 2. Agregace podle některých sloupců

---

Necháme si vypsat unikátní hodnoty pro hledané sloupce:

In [ ]:
novy_df['PRODUCT'].unique()

In [ ]:
novy_df['COUNTRY'].unique()

In [ ]:
novy_df.head(2)

In [ ]:
novy_df.groupby('COUNTRY')

In [ ]:
zkraceny_novy_df = novy_df[['COUNTRY', 'OFFERED_CAPACITY_PRICE_NEW']]

<br>

Opatrně na seskupování, tak aby data dávala nějaký smysl:

In [ ]:
zkraceny_novy_df.groupby('COUNTRY').max()

In [ ]:
dalsi_zkraceny_novy_df = novy_df[['COUNTRY', 'PRODUCT', 'OFFERED_CAPACITY_PRICE_NEW', 'SETTLEMENTCAPACITY_PRICE_NEW']]

In [ ]:
dalsi_zkraceny_novy_df

In [ ]:
vystup = dalsi_zkraceny_novy_df.pivot_table(values='OFFERED_CAPACITY_PRICE_NEW',
                                   index='COUNTRY', columns='PRODUCT').round(1).unstack()

In [ ]:
vystup.index

In [ ]:
import numpy as np

In [ ]:
agregacni_predpis = {'OFFERED_CAPACITY_PRICE_NEW': np.median}

In [ ]:
dalsi_zkraceny_novy_df.pivot_table(values='OFFERED_CAPACITY_PRICE_NEW',
                                   index='COUNTRY', columns='PRODUCT',
                                   aggfunc=agregacni_predpis).round(1)

---


<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.odqbr-09dxxBQjF7x7NyBAHaII%26pid%3DApi&f=1&ipt=5b4a03c1dd38b52108237102ebb350595a5d6ea6a50572f5897ca89fbf5b5445&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

<br>

## Časové řady

---

Část *frameworku* byla vyvinuta za účelem **finančního modelování**.

Proto je více než dobře vybavena sadou nástrojů, které umí pracovat **s daty, časem a časovými objekty**.

<br>

Uplatnění:
* **Analýza a predikce**: Časové řady umožňují analyzovat vývoj hodnot v čase a předpovídat budoucí trendy nebo sezónní změny. To je klíčové v oblastech jako finance, ekonomika, prodej, počasí a mnoho dalších.

* **Efektivní manipulace s časem**: Pandas poskytuje nástroje pro efektivní práci s časovými řadami, což zjednodušuje úkoly, jako je agregace, interpolace nebo časové posuny. Díky integrovaným funkcím můžete snadno pracovat s daty různých časových frekvencí a konvertovat mezi nimi.

* **Časově závislé analýzy**: Časové řady umožňují provádět časově závislé analýzy, jako je detekce změn v trendech, identifikace sezónních vlivů nebo identifikace časových závislostí mezi proměnnými.

* **Flexibilní indexace**: Pandas podporuje flexibilní indexaci časových řad, což usnadňuje filtrování, řazení a výběr dat na základě časových značek nebo rozsahů.

* **Kompatibilita s dalšími knihovnami**: Pandas je kompatibilní s mnoha dalšími knihovnami pro analýzu časových řad, jako je statsmodels nebo scikit-learn. To usnadňuje integraci a rozšíření vašich analýz s pokročilejšími metodami a algoritmy.
    
<br>

Jde například o údaje typu:
* *timestampy*, údaj odkazující na konkrétní **časový okamžik** (např. `26. července 2024 v 14:36 hod.`),
* *časové intervaly*, tedy období odkazují na délku času **mezi konkrétním začátkem a koncem** (např. intervaly ze dne na den),
* *time delta* objekty, tedy přesné délky času (např. 22,22 sekundy).

#### Data a čas v Pythonu

---

Standardní výbavou Pythonu jsou knihovny `datetime`:

In [ ]:
from datetime import datetime

In [ ]:
datetime(year=2023, month=4, day=5)

<br>

Nebo knihovna `dateutil` pro parsování datových typů z různých stringových zadání:

In [ ]:
from dateutil import parser

In [ ]:
date = parser.parse("26th of november, 2024")

In [ ]:
date

Kde pomocí metody `strftime` můžeš vypsat den:

In [ ]:
date.strftime("%A")

#### Data a čas v numpy

---

Některé nedostatky uvnitř knihoven `datetime` a `dateutil` vedli ke vzniku sady nástrojů.

Tyto doplňky vznikly pod hlavičkou knihovny `numpy`.

In [ ]:
import numpy as np
from numpy import array, arange

In [ ]:
date = array('2023-04-05', dtype=np.datetime64)

In [ ]:
date

<br>

Pokud potřebuješ pole následujících 7 dní:

In [ ]:
date + arange(7)

<br>

Vzhledem k jednotnému datovu typu v poli pro **numpy** `datetime64` může tento typ operace
provádět mnohem rychleji, než přímo v Pythonu `datetime` objekty, zejména když objekty nabývají na velikosti.

#### Data a čas v pandách

---

Jde o kombinace objektů z obou předchozích podkapitol.

Ty dávají dohromady to nejlepší prostředky pro zacházení s časem.

In [ ]:
from pandas import to_datetime

In [ ]:
date = to_datetime("5th of April, 2023")

In [ ]:
date

In [ ]:
date.strftime("%A")

## Časové řady

---

V podstatě jde o hlavní nástroj, který tato knihovna dovede nabídnout.

#### Indexování časem
---
`DatetimeIndex` obsahuje časové značky (*timestamp*), které jsou uloženy ve formátu `datetime64` s nanosekundovou přesností.

Tento objekt umožňuje efektivní práci s časovými řadami a poskytuje mnoho funkcí pro manipulaci s daty a časy.

Výhody práce s `DatetimeIndex` objektem:
* Časové zóny,
* frekvence,
* časově závislé selekce,
* atributy časových značek,
* operace s časem.

In [ ]:
from pandas import DatetimeIndex, Series, to_datetime

In [ ]:
datumy = ["2023-04-05", "2022-04-05", "2021-04-05", "2020-04-05"]

In [ ]:
indexy = DatetimeIndex(datumy)

In [ ]:
hodnoty = [to_datetime(den).strftime("%A") for den in datumy]

In [ ]:
hodnoty

In [ ]:
hodnoty_sl = Series(hodnoty, index=indexy)

In [ ]:
hodnoty_sl

In [ ]:
hodnoty_sl.index

In [ ]:
type(indexy)

<br>

Můžeš vytvořit selektivní proces na základě datumu pro index:

In [ ]:
hodnoty_sl[:'2021-04-05']

<br>

## Základní objekty

---

Mezi základní objekty pro práci s časem patří:
* `Timestamp` typ (související struktura Indexu `DatetimeIndex`,
* `Period` typ (.. `PeriodIndex`),
* `Timedelta` typ (.. `TimedeltaIndex`).

#### Timestamp & DatetimeIndex

---

Nejčastější datové typy, které lze vyvolat přímo, ovšem běžnější je pracovat s funkcí `to_datetime`.

Funkce `to_datetime` umí parsovat různé stringové formáty.

<br>

### Práce s jedním datumem

---

In [ ]:
datum_objekt = to_datetime("05/04/2023")  # DD/MM/RRRR, 5. duben 2023

<br>

Opatrně na specifický formát datumu:

In [ ]:
datum_objekt                              # MM/DD/RRRR

In [ ]:
type(datum_objekt)

In [ ]:
datum_objekt.month_name()

Pokud do funkce `to_datetime` vložíš jedinou hodnotu, vrací objekty typu `Timestamp`.

<br>

#### Specifický formát

---

Pokud se *parser* ztratí nebo tvoje zadání neodpovídá jeho vyhotovení:

In [ ]:
to_datetime("12-11-2010 00:00", format="%d-%m-%Y %H:%M")

In [ ]:
naformatovany_datum = to_datetime("05/04/2023", format='%d/%m/%Y')

In [ ]:
naformatovany_datum.month_name()

In [ ]:
naformatovany_datum.day_name()

Nespoléhat na defaultní nastavení funkce `to_datetime`, ale umět si formát upravit podle svých potřeb.

<br>

### Práce s několika datumy

---

In [ ]:
datumy = to_datetime([datetime(2023, 4, 5),
                      "5th of April 2023",
                      "2023-Apr-5",
                      "05-04-2023",
                      "20230405"])

In [ ]:
datumy

In [ ]:
type(datumy)

Zatímco pole hodnot s datumy, které funkce `to_datetime` zpracuje vrací objekt typu `DatetimeIndex`.

<br>

Objekt `DatetimeIndex` potom můžeš konvertovat na `PeriodIndex` pomocí *metody* `to_period`:

In [ ]:
# datumy.to_period?

<br>

Tato metoda je užitečná, pokud chcete převést časové řady **na určité časové období**.

Když budeš třeba potřebovat převést hodnoty **z denních dat na měsíční data**.

<br>

#### Denní data

---

In [ ]:
datumy.to_period(freq="D")

<br>

#### Měsíční data

---

In [ ]:
from pandas import DataFrame
import pandas as pd

In [ ]:
datumy.to_period(freq="M")

<br>

V rámci periody jsem schopen u časových řad nastavovat, nebo měnit časové intervaly (tzn. denní, měsíční).

In [ ]:
# data = {'date': pd.date_range(start='2022-01-01', periods=10, freq='ME'),
#         'sales': [100, 102, 105, 107, 110, 112, 115, 117, 120, 122]}

<br>

Varianta frekvence `ME` lze použít ve verzi pandas 2.2.

Pro starší varianty je nutné pracovat s argumentem `M`.

In [ ]:
data = {'date': pd.date_range(start='2022-01-01', periods=10, freq='M'),
        'sales': [100, 102, 105, 107, 110, 112, 115, 117, 120, 122]}

In [ ]:
data_df = DataFrame(data)

In [ ]:
data_df

<br>

Index původní tabulky je nastavený implicitně jako interval `0-n` .

In [ ]:
data_df.set_index('date', inplace=True)

In [ ]:
data_df

In [ ]:
period_df = data_df.to_period(freq='M')

In [ ]:
period_df

<br>

### TimedeltaIndex

---

Jde o rozdíl v jednotkách času.

Nejčastěji se s tímto objektem setkáš, pokud potřebuješ získat **rozdíl mezi dvěma datumy**.

In [ ]:
from datetime import datetime

from pandas import to_datetime

In [ ]:
specialni_datumy = to_datetime([datetime(2022, 4, 5),
                                "5th of April 2021",
                                "2020-Apr-5",
                                "05-04-2019",
                                "20180405"])

In [ ]:
specialni_datumy

In [ ]:
datumy

In [ ]:
datumy - specialni_datumy

<br>

Rozdíl mezi datumy můžeš počítat i pro jednotlivé `Timestamp` hodnoty:

In [ ]:
datumy[0]

In [ ]:
specialni_datumy[0]

In [ ]:
datumy[0] - specialni_datumy[0]

In [ ]:
datovy_objekt = to_datetime('2022-04-05')

In [ ]:
datovy_objekt.tzname()

In [ ]:
aktualni_cas = pd.Timestamp('2017-01-01T12')

In [ ]:
aktualni_cas

<br>

Pokud chybí v objektu informace o pásmu, je možnost ji doplnit:

In [ ]:
aktualni_cas = aktualni_cas.tz_localize('CET')

In [ ]:
aktualni_cas

In [ ]:
aktualni_cas.tzname()

In [ ]:
nova_hodnota = pd.Timestamp('2017-01-01 12:00:00+0200')

In [ ]:
nova_hodnota.tzname()

In [ ]:
nova_hodnota.tz

<br>

Pro převod univerzální hodnoty slouží metoda `tz_convert`.

<br>

#### 🧠 CVIČENÍ 🧠, procvič si funkci `to_datetime`

---

In [ ]:
# Převeď datumy na formát RRRR-MM-DD
data = {"Name": ["Event A", "Event B", "Event C"],
        "Date": ["21-11-2024", "05-12-2024", "15-01-2025"]}

In [ ]:
data_df = DataFrame(data, index=data['Date'])

In [ ]:
data_df

In [ ]:
data_df['Date'] = to_datetime(data_df['Date'], format='%d-%m-%Y')

In [ ]:
data_df

<details>
    <summary>▶️ Řešení</summary>
    
```python
import pandas as pd

data = {
    "Name": ["Event A", "Event B", "Event C"],
    "Date": ["21-11-2024", "05-12-2024", "15-01-2025"],
}


df = pd.DataFrame(data)


def convert_dates(df):
    df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y")
    return df

converted_df = convert_dates(df)
print(converted_df)
print(converted_df.dtypes)
```
</details>

<br>

## Funkce `date_range`

---

Aby bylo zadání řady (sekvence) dat pohodlnější, vyzkoušej funkci `date_range`.

Obdobně potom pracují související funkce:
* `date_range`, timestampy,
* `period_range`, periody,
* `timedelta_range`, pro delty.

In [ ]:
from pandas import date_range

<br>

### Počet period

---

In [ ]:
datumy_rada = date_range(start="01-01-1992", periods=5, freq='10min15s')

<br>

Specifikace hodnoty frekvence vychází vždy z tabulky [offsetů](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).

In [ ]:
datumy_rada

<br>

## Frekvence

---

In [ ]:
datumy_mesicne_ind = date_range("01-01-1992", periods=12, freq="M")

In [ ]:
datumy_mesicne_ind

<br>

### Hodinové periody

---

In [ ]:
from pandas import timedelta_range

In [ ]:
hodinove_ind = timedelta_range(0, periods=12, freq='H')

In [ ]:
hodinove_ind

In [ ]:
# timedelta_range?

<br>

### Tabulka frekvencí

---

Časovou paletou frekvencí, kterou framework `pandas` nabízí je tato tabulka:

| String | Popisek |
| :-: | :- |
| `D` | kalendářní den |
| `W` | týden |
| `M` | konec měsíce |
| `Q` | konec čtvrtletí |
| `A` | konec roku |
| `H` | hodiny |
| `T` | minuty |
| `S` | vteřiny |
| `B` | pracovní den |
| `BM` | konec pracovního měsíce |
| `BQ` | konec pracovního čtvrtletí |

#### Netradiční frekvence

---

In [ ]:
timedelta_range(start='1H', periods=5, freq="1H15T")

<br>

### Resampling

---

*Resampling* nebo také *převzorkování* je proces, který upravuje frekvenci časové řady.

*Resampling* se obvykle používá **pro snížení frekvence** (z hodinových dat na denní data).

*Resampling* zahrnuje **agregaci dat**.

In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, date_range

<br>

V `pandas` se k tomu používá metoda `resample`, která má jako parametr novou frekvenci, na kterou chcete data převést:

In [2]:
data = {'date': pd.date_range(start='2022-01-01', periods=60, freq='D'),
        'sales': np.random.randint(10, 100, size=60)}

In [3]:
data_df = pd.DataFrame(data)

In [5]:
data_df.head(7)

,date,sales
0,2022-01-01,77
1,2022-01-02,89
2,2022-01-03,78
3,2022-01-04,73
4,2022-01-05,24
5,2022-01-06,11
6,2022-01-07,27


Dostupné sloupce:

In [6]:
data_df.columns

Index(['date', 'sales'], dtype='object')

Dostupné indexy:

In [7]:
data_df.index

RangeIndex(start=0, stop=60, step=1)

<br>

Nastavení nového sloupce pro Index:

In [8]:
data_df.set_index('date', inplace=True)

In [9]:
data_df.head()

,sales
date,
2022-01-01,77
2022-01-02,89
2022-01-03,78
2022-01-04,73
2022-01-05,24


<br>

Pomocí metody `resample` současně hodnoty agregujeme a snížíme frekvenci:

In [13]:
pd.__version__

'2.0.3'

In [20]:
mesicni_data_df = data_df.resample('M').sum()  # 'M'/'ME' - end of the month

In [21]:
mesicni_data_df

,sales
date,
2022-01-31,1564
2022-02-28,1760
2022-03-31,18


In [22]:
data_df.tail()

,sales
date,
2022-02-25,96
2022-02-26,34
2022-02-27,85
2022-02-28,59
2022-03-01,18


<br>

Upravit původní vzorek dat na zamýšlenou frekvenci, tedy `W`:

In [42]:
tydenni_data_df = data_df.resample('W').mean().round(1)

In [43]:
tydenni_data_df

,sales
date,
2022-01-02,83.0
2022-01-09,44.3
2022-01-16,62.6
2022-01-23,41.4
2022-01-30,39.4
2022-02-06,67.0
2022-02-13,67.0
2022-02-20,60.7
2022-02-27,60.3


In [26]:
from pandas import to_datetime

In [31]:
to_datetime('2022-01-02').day_name()

'Sunday'

<br>

### Zpřesňování

---

Zahrnuje **zvýšení frekvence časových řad**.

Přičemž se nově vytvořené hodnoty obvykle interpolují nebo doplňují nějakou konstantou.

V pandas se k zpřesňování používá metoda `asfreq`, která má jako parametr **novou frekvenci**, na kterou chcete data převést.

In [44]:
data = {'date': pd.date_range(start='2022-01-01', end='2022-03-01', freq='D'),
        'sales': np.random.randint(10, 150, size=60)}  # Opatrně na délky sekvencí

In [45]:
dalsi_data_df = pd.DataFrame(data)

In [46]:
dalsi_data_df.head()

,date,sales
0,2022-01-01,40
1,2022-01-02,139
2,2022-01-03,113
3,2022-01-04,126
4,2022-01-05,40


In [47]:
dalsi_data_df.set_index('date', inplace=True)

In [50]:
dalsi_data_df.head(10)

,sales
date,
2022-01-01,40
2022-01-02,139
2022-01-03,113
2022-01-04,126
2022-01-05,40
2022-01-06,85
2022-01-07,84
2022-01-08,71
2022-01-09,19


In [48]:
denni_data = dalsi_data_df.asfreq('H')

<br>

Pokud nezadáš hodnotu pro výplňovací parametr, uvidíš prázdné hodnoty `NaN`:

In [52]:
denni_data.head(25)

,sales
date,
2022-01-01 00:00:00,40.0
2022-01-01 01:00:00,NaN
2022-01-01 02:00:00,NaN
2022-01-01 03:00:00,NaN
2022-01-01 04:00:00,NaN
2022-01-01 05:00:00,NaN
2022-01-01 06:00:00,NaN
2022-01-01 07:00:00,NaN
2022-01-01 08:00:00,NaN


Metoda `asfreq` bere jako parametr novou frekvenci, na kterou chceme data převést.

V případě výše jde o frekvenci `H`, což znamená **hodinová data**.

Protože při zpřesňování se **vytvoří chybějící hodnoty** (v důsledku zvýšení frekvence), je potřeba zvolit metodu pro jejich doplnění.

In [53]:
denni_data_df = dalsi_data_df.asfreq('H', method='ffill')  # ffill, bfill

In [56]:
denni_data_df.head(26)

,sales
date,
2022-01-01 00:00:00,40
2022-01-01 01:00:00,40
2022-01-01 02:00:00,40
2022-01-01 03:00:00,40
2022-01-01 04:00:00,40
2022-01-01 05:00:00,40
2022-01-01 06:00:00,40
2022-01-01 07:00:00,40
2022-01-01 08:00:00,40


Výš jde o metodu `forward fill` (parametr `method='ffill'`), která kopíruje předchozí hodnotu pro výplň chybějících hodnot.

Další možností je použít metodu `'back fill'` (parametr `method='bfill'`), která kopíruje následující hodnotu pro výplň chybějících hodnot.

In [57]:
denni_data = dalsi_data_df.asfreq('H', method='bfill')

In [58]:
denni_data.head(25)

,sales
date,
2022-01-01 00:00:00,40
2022-01-01 01:00:00,139
2022-01-01 02:00:00,139
2022-01-01 03:00:00,139
2022-01-01 04:00:00,139
2022-01-01 05:00:00,139
2022-01-01 06:00:00,139
2022-01-01 07:00:00,139
2022-01-01 08:00:00,139


In [81]:
nova_rada_prodeju = (102,
 72,
 118,
 108,
 87,
 140,
 122,
 110,
 10,
 102,
 10,
 139,
 148,
 86,
 37,
 66,
 84,
 127,
 24,
 114,
 20,
 102,
 66,
 29,
 141,
 50,
 36,
 60,
 137,
 104,
 145,
 147,
 117,
 118,
 10,
 112,
 55,
 43,
 16,
 112,
 69,
 136,
 25,
 91,
 111,
 59,
 13,
 114,
 34,
 63,
 91,
 82,
 30,
 77,
 145,
 133,
 48,
 61,
 103)

In [62]:
data = {'date': pd.date_range(start='2022-01-01', end='2022-03-01', freq='D'),
        'sales': nova_rada_prodeju}

In [64]:
data_df = DataFrame(data)

In [65]:
data_df.set_index('date', inplace=True)

In [66]:
data_df.head()

,sales
date,
2022-01-01,102.0
2022-01-02,NaN
2022-01-03,72.0
2022-01-04,118.0
2022-01-05,108.0


In [77]:
tuple(pd.date_range(start='2022-01-01', end='2022-03-01', freq='D'))

(Timestamp('2022-01-01 00:00:00'),
 Timestamp('2022-01-02 00:00:00'),
 Timestamp('2022-01-03 00:00:00'),
 Timestamp('2022-01-04 00:00:00'),
 Timestamp('2022-01-05 00:00:00'),
 Timestamp('2022-01-06 00:00:00'),
 Timestamp('2022-01-07 00:00:00'),
 Timestamp('2022-01-08 00:00:00'),
 Timestamp('2022-01-09 00:00:00'),
 Timestamp('2022-01-10 00:00:00'),
 Timestamp('2022-01-11 00:00:00'),
 Timestamp('2022-01-12 00:00:00'),
 Timestamp('2022-01-13 00:00:00'),
 Timestamp('2022-01-14 00:00:00'),
 Timestamp('2022-01-15 00:00:00'),
 Timestamp('2022-01-16 00:00:00'),
 Timestamp('2022-01-17 00:00:00'),
 Timestamp('2022-01-18 00:00:00'),
 Timestamp('2022-01-19 00:00:00'),
 Timestamp('2022-01-20 00:00:00'),
 Timestamp('2022-01-21 00:00:00'),
 Timestamp('2022-01-22 00:00:00'),
 Timestamp('2022-01-23 00:00:00'),
 Timestamp('2022-01-24 00:00:00'),
 Timestamp('2022-01-25 00:00:00'),
 Timestamp('2022-01-26 00:00:00'),
 Timestamp('2022-01-27 00:00:00'),
 Timestamp('2022-01-28 00:00:00'),
 Timestamp('2022-01-

In [79]:
from pandas import Timestamp

In [80]:
nova_casova_rada = (Timestamp('2022-01-01 00:00:00'),
 Timestamp('2022-01-03 00:00:00'),
 Timestamp('2022-01-04 00:00:00'),
 Timestamp('2022-01-05 00:00:00'),
 Timestamp('2022-01-06 00:00:00'),
 Timestamp('2022-01-07 00:00:00'),
 Timestamp('2022-01-08 00:00:00'),
 Timestamp('2022-01-09 00:00:00'),
 Timestamp('2022-01-10 00:00:00'),
 Timestamp('2022-01-11 00:00:00'),
 Timestamp('2022-01-12 00:00:00'),
 Timestamp('2022-01-13 00:00:00'),
 Timestamp('2022-01-14 00:00:00'),
 Timestamp('2022-01-15 00:00:00'),
 Timestamp('2022-01-16 00:00:00'),
 Timestamp('2022-01-17 00:00:00'),
 Timestamp('2022-01-18 00:00:00'),
 Timestamp('2022-01-19 00:00:00'),
 Timestamp('2022-01-20 00:00:00'),
 Timestamp('2022-01-21 00:00:00'),
 Timestamp('2022-01-22 00:00:00'),
 Timestamp('2022-01-23 00:00:00'),
 Timestamp('2022-01-24 00:00:00'),
 Timestamp('2022-01-25 00:00:00'),
 Timestamp('2022-01-26 00:00:00'),
 Timestamp('2022-01-27 00:00:00'),
 Timestamp('2022-01-28 00:00:00'),
 Timestamp('2022-01-29 00:00:00'),
 Timestamp('2022-01-30 00:00:00'),
 Timestamp('2022-01-31 00:00:00'),
 Timestamp('2022-02-01 00:00:00'),
 Timestamp('2022-02-02 00:00:00'),
 Timestamp('2022-02-03 00:00:00'),
 Timestamp('2022-02-04 00:00:00'),
 Timestamp('2022-02-05 00:00:00'),
 Timestamp('2022-02-06 00:00:00'),
 Timestamp('2022-02-07 00:00:00'),
 Timestamp('2022-02-08 00:00:00'),
 Timestamp('2022-02-09 00:00:00'),
 Timestamp('2022-02-10 00:00:00'),
 Timestamp('2022-02-11 00:00:00'),
 Timestamp('2022-02-12 00:00:00'),
 Timestamp('2022-02-13 00:00:00'),
 Timestamp('2022-02-14 00:00:00'),
 Timestamp('2022-02-15 00:00:00'),
 Timestamp('2022-02-16 00:00:00'),
 Timestamp('2022-02-17 00:00:00'),
 Timestamp('2022-02-18 00:00:00'),
 Timestamp('2022-02-19 00:00:00'),
 Timestamp('2022-02-20 00:00:00'),
 Timestamp('2022-02-21 00:00:00'),
 Timestamp('2022-02-22 00:00:00'),
 Timestamp('2022-02-23 00:00:00'),
 Timestamp('2022-02-24 00:00:00'),
 Timestamp('2022-02-25 00:00:00'),
 Timestamp('2022-02-26 00:00:00'),
 Timestamp('2022-02-27 00:00:00'),
 Timestamp('2022-02-28 00:00:00'),
 Timestamp('2022-03-01 00:00:00'))

In [82]:
data = {'date': nova_casova_rada,
        'sales': nova_rada_prodeju}

In [83]:
novy_df = DataFrame(data)

In [84]:
novy_df.set_index('date', inplace=True)

In [85]:
novy_df.head()

,sales
date,
2022-01-01,102
2022-01-03,72
2022-01-04,118
2022-01-05,108
2022-01-06,87


In [93]:
mesicni_data_asfreq = novy_df.asfreq('H', method='ffill')  # fillna

In [95]:
mesicni_data_asfreq.head(10)

,sales
date,
2022-01-01 00:00:00,102
2022-01-01 01:00:00,50
2022-01-01 02:00:00,50
2022-01-01 03:00:00,50
2022-01-01 04:00:00,50
2022-01-01 05:00:00,50
2022-01-01 06:00:00,50
2022-01-01 07:00:00,50
2022-01-01 08:00:00,50


<br>

Pokud mi nevyhovují metody `bfill` a `ffill`, tak můžu nově vzniklé záznamy vyplnit konstantou `fill_value`.

<br>

**🧠 CVIČENÍ 🧠, procvič si časové řady**

Analyzuj prodeje produktu ve fiktivní společnosti během jednoho roku a zjisti následující:
1. Celkový prodej za každý měsíc.
2. Průměrný prodej za každý den v týdnu.
3. Denní prodej za poslední týden.

In [96]:
import pandas as pd
import numpy as np

In [125]:
denni_datumy = pd.date_range(start='2022-01-01',
                             end='2022-12-31',
                             freq='D')
prodeje = np.random.randint(10, 100, size=(len(denni_datumy),))

In [126]:
df_prodeje = pd.DataFrame({'datumy': denni_datumy, 'prodeje': prodeje})

In [101]:
df_prodeje.set_index('datumy', inplace=True)

In [127]:
df_prodeje.head()

,datumy,prodeje
0,2022-01-01,58
1,2022-01-02,62
2,2022-01-03,49
3,2022-01-04,89
4,2022-01-05,77


In [110]:
r1 = df_prodeje.resample('M').sum().to_period(freq='M')

In [115]:
r1

,prodeje
datumy,
2022-01,1601
2022-02,1429
2022-03,1964
2022-04,1995
2022-05,1800
2022-06,1768
2022-07,1563
2022-08,1733
2022-09,1533


In [120]:
df_prodeje

,prodeje
datumy,
2022-01-01,59
2022-01-02,73
2022-01-03,53
2022-01-04,59
2022-01-05,82
...,...
2022-12-27,88
2022-12-28,97
2022-12-29,18


In [123]:
from pandas import to_datetime

In [138]:
to_datetime(df_prodeje['datumy']).dt.strftime('%Y/%m')

0      2022/01
1      2022/01
2      2022/01
3      2022/01
4      2022/01
        ...   
360    2022/12
361    2022/12
362    2022/12
363    2022/12
364    2022/12
Name: datumy, Length: 365, dtype: object

In [139]:
# TODO: Doplnit převedené do řešení.

<details>
    <summary>▶️ Řešení</summary>
    
```python
monthly_sales = df.resample('M').sum()
print("Celkový prodej za každý měsíc:")
print(monthly_sales)

df_prodeje['weekday'] = df_prodeje.index.weekday
average_weekday_sales = df_prodeje.groupby('weekday')['prodeje'].mean()
print("\nPrůměrný prodej za každý den v týdnu:")
print(average_weekday_sales)

last_week_sales = df.loc['2022-12-25':, 'sales']
print("\nDenní prodej za poslední týden:")
print(last_week_sales)
```
</details>

---